In [3]:
# Multivariate Time-Series Forecasting – Data Loading & Preprocessing
"""
This notebook covers:
- Loading raw Walmart sales datasets
- Merging sales, store, and feature data
- Handling missing values and data quality issues
- Creating a clean time-indexed dataset for further analysis
"""
print("This notebook is intended to be run in a Jupyter environment.")

This notebook is intended to be run in a Jupyter environment.


In [4]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)


In [5]:
train_path = "C:\\Multivariate_TimeSeries_Forecasting_CP2\\data\\raw\\train.csv"
features_path = "C:\\Multivariate_TimeSeries_Forecasting_CP2\\data\\raw\\features.csv"
stores_path = "C:\\Multivariate_TimeSeries_Forecasting_CP2\\data\\raw\\stores.csv"

train_df = pd.read_csv(train_path)
features_df = pd.read_csv(features_path)
stores_df = pd.read_csv(stores_path)

print(train_df.shape, features_df.shape, stores_df.shape)


(421570, 5) (8190, 12) (45, 3)


In [6]:
train_df.head()
features_df.head()
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [7]:
train_df.info()
features_df.info()
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4  

In [9]:
# Convert "Date" columns to datetime
train_df["Date"] = pd.to_datetime(train_df["Date"])
features_df["Date"] = pd.to_datetime(features_df["Date"])


In [10]:
# Merge datasets
merged_df = pd.merge(
    train_df,
    features_df,
    how="left",
    on=["Store", "Date", "IsHoliday"]
)

merged_df.shape


(421570, 14)

In [12]:
# Final merge with stores data
merged_df = pd.merge(
    merged_df,
    stores_df,
    how="left",
    on="Store"
)

merged_df.shape
merged_df.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type_x,Size_x,Type_y,Size_y
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315,A,151315


In [13]:
# Check for missing values
merged_df.isna().sum().sort_values(ascending=False)


MarkDown2       310322
MarkDown4       286603
MarkDown3       284479
MarkDown1       270889
MarkDown5       270138
Store                0
Dept                 0
IsHoliday            0
Temperature          0
Date                 0
Fuel_Price           0
Weekly_Sales         0
CPI                  0
Unemployment         0
Type_x               0
Size_x               0
Type_y               0
Size_y               0
dtype: int64

In [14]:
# Fill missing values in MarkDown columns with 0
markdown_cols = ["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]

merged_df[markdown_cols] = merged_df[markdown_cols].fillna(0)



In [ ]:
# Handle Remaining Numerical Missing Values
merged_df["CPI"] = merged_df["CPI"].fillna(method="ffill")
merged_df["Unemployment"] = merged_df["Unemployment"].fillna(method="ffill")



C:\Users\jessy\AppData\Local\Temp\ipykernel_15340\1585616620.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["CPI"] = merged_df["CPI"].fillna(method="ffill")
C:\Users\jessy\AppData\Local\Temp\ipykernel_15340\1585616620.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["Unemployment"] = merged_df["Unemployment"].fillna(method="ffill")


In [16]:
merged_df.isna().sum()


Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
Type_x          0
Size_x          0
Type_y          0
Size_y          0
dtype: int64

In [17]:
# Sort Data by Time
merged_df = merged_df.sort_values(["Store", "Dept", "Date"])
merged_df.head()


,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type_x,Size_x,Type_y,Size_y
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,A,151315,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,A,151315,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,A,151315,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,A,151315,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,A,151315,A,151315


In [18]:
# Set Date as Index
merged_df.set_index("Date", inplace=True)
merged_df.head()


,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type_x,Size_x,Type_y,Size_y
Date,,,,,,,,,,,,,,,,,
2010-02-05,1,1,24924.50,False,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,A,151315,A,151315
2010-02-12,1,1,46039.49,True,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,A,151315,A,151315
2010-02-19,1,1,41595.55,False,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,A,151315,A,151315
2010-02-26,1,1,19403.54,False,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,A,151315,A,151315
2010-03-05,1,1,21827.90,False,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,A,151315,A,151315


In [20]:
# Save Cleaned Dataset 
processed_path = "C:\\Multivariate_TimeSeries_Forecasting_CP2\\data\\raw\\processed\\walmart_merged_cleaned.csv"
merged_df.to_csv(processed_path)

print("Cleaned dataset saved successfully.")


Cleaned dataset saved successfully.
